Dataset Source : https://www.kaggle.com/grassknoted/asl-alphabet

Refrence : https://www.kaggle.com/ryuodan/asl-detection-walkthrough

### Mengorganisir File dan Folder 

In [ ]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download grassknoted/asl-alphabet

In [ ]:
import zipfile, os
local_zip = '/content/asl-alphabet.zip'

# Buat dataset folder
zip_target = '/content/'
# !mkdir /content/zip_target/

# Unzip dataset
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(zip_target)
zip_ref.close()

In [ ]:
!mkdir /content/val

In [ ]:
import random, shutil

# Membuat validation dan test folder dan data

base = '/content/asl_alphabet_train/asl_alphabet_train'
base_dest = '/content/val'
base_subdirs = list()
for y in os.listdir(base):
  base_subdirs.append(y)

unused_var, num = None, None
for unused_var in base_subdirs:
  if os.path.exists(os.path.join(base_dest, unused_var)):
    continue
  else:
    os.makedirs(os.path.join(base_dest, unused_var))

# memindahkan sample file dari subdir-subdir yang berada di base_dir
for subdir in base_subdirs:
  source = os.path.join(base, subdir)
  fin_dest = os.path.join(base_dest, subdir)
  files = os.listdir(source)
  no_of_files = len(files) * 0.2

  for file_name in random.sample(files, int(round(no_of_files))):
      shutil.move(os.path.join(source, file_name), fin_dest)
unused_var, num = None, None

### Train AI

In [ ]:
# base_dir = '/content/dataset/images'
train_dir = '/content/asl_alphabet_train/asl_alphabet_train'
validation_dir = '/content/val'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale         = 1./255,
                    rotation_range  = 20,
                    horizontal_flip = True,
                    shear_range     = 0.2,
                    fill_mode       = 'nearest'
                )

test_datagen  = ImageDataGenerator(
                    rescale         = 1./255,
                    rotation_range  = 20,
                    horizontal_flip = True,
                    shear_range     = 0.2,
                    fill_mode       = 'nearest'
                )

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (64,64),
    batch_size  = 32,
    class_mode  = 'categorical'
)

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size = (64,64),
    batch_size  = 32,
    class_mode  = 'categorical'
)

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(32, (5,5), activation='relu', input_shape=(64,64,3)),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    # tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
                                    # tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation='relu'),
                                    tf.keras.layers.Dense(29, activation='softmax')
])

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [ ]:
model.fit(
  train_generator,
  # steps_per_epoch   = 25,
  epochs            = 7,
  validation_data   = validation_generator
  # validation_steps  = 5,
  # verbose           = 2
)

In [ ]:
model.save('ASL_Alphabet.h5')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing import image
%matplotlib inline

# prediction image
# path = '/content/asl_alphabet_test/asl_alphabet_test/C_test.jpg'
# path = '/content/asl_alphabet_test/asl_alphabet_test/A_test.jpg'
# path = '/content/asl_alphabet_test/asl_alphabet_test/O_test.jpg'
path = '/content/asl_alphabet_test/asl_alphabet_test/P_test.jpg'

img = image.load_img(path, target_size = (64,64))
imgplot = plt.imshow(img)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=20)
print(path)
list_of_letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'del', 'nothing', 'space']

test__ = np.array(classes[0])
print("Result : " + str(list_of_letters[np.argmax(test__)]))